In [2]:
import numpy as np
import pandas as pd
import heapq
import random

# Generation of initial population of random antibodies

In [3]:
# m: number of machines
# p: number of parts
# population_size : number of antibody of the initial population
# seed: seed for reproducible results
def generation_initial_population(p, m, population_size, seed):
    MaxCell = min(p,m) #calculation of max number of cells
    number_of_zeros = MaxCell - 1 #number of zeros in each antibody
    antibodies = np.empty(shape=(population_size, p+m+number_of_zeros), dtype=int)
    antibody = np.append(np.array([*range(1,p+m+1)]), np.zeros(number_of_zeros,dtype=int))
    for i in range(0,population_size):
        np.random.seed(seed) 
        np.random.shuffle(antibody) #random positions in the array
        antibodies[i] = antibody
    return antibodies

In [31]:
antibodies = generation_initial_population(p = 7, m = 5, population_size = 3, seed = 1995)
antibodies

array([[ 1,  8, 11,  0,  0, 12,  0,  9, 10,  0,  6,  2,  4,  7,  3,  5],
       [ 1,  9,  6,  3,  4,  2,  7, 10,  0,  5, 12,  8,  0,  0, 11,  0],
       [ 1, 10, 12, 11,  0,  8,  0,  0,  5,  0,  2,  9,  3,  7,  6,  4]])

# Evaluate all existing antibodies and compute their affinities

Lectura de los datos del problema + Traducción del formato del dataset a matriz trabajo-estación

In [5]:
def part_machine_incidence_matrix(data):    
    f = open(data,'r')
    lines = [line.split('\n') for line in f]
    f.close()

    m,p = [int(num) for num in lines[8][0].split(' ')] #m: number of machines, p: number of parts

    machines=[[m,p]]
    for i in range(9,9+m):
        #machines.append([int(num) for num in lines[i][0].split(' ')])
        machines.append([int(lines[i][0].split(' ')[j]) for j in range(1,m)])

    columns, rows = ['M'+str(i) for i in range(1,m+1)], ['P'+str(i) for i in range(1,p+1)]
    m_p_matrix = pd.DataFrame(columns= columns, index= rows)

    ones_zeros = []
    for i in range(1,len(machines)):
        aux = []
        for j in range(1,p+1):
            if j in machines[i]: aux.append(1)
            else: aux.append(0)
        ones_zeros.append(aux)

    for i in range(0,len(columns)):
        m_p_matrix[columns[i]] = ones_zeros[i]

    return m_p_matrix, m, p, columns, rows

In [6]:
matrix, m, p, columns, rows = part_machine_incidence_matrix('data/instances/testset_a/5x7_Waghodekar_Sahu(1984)[Problem-2].txt')
matrix

,M1,M2,M3,M4,M5
P1,1,0,0,1,0
P2,0,1,0,1,1
P3,0,1,1,1,0
P4,0,1,1,1,1
P5,1,1,1,0,1
P6,1,0,1,0,1
P7,1,0,0,0,0


## Decodificación de anticuerpos
1er Paso: Separación de celdas.

In [33]:
def cell_identification(antibodies):
    total_cells = []
    for antibodie in antibodies:
        print("antibodie", antibodie)
        flag = 1 #bandera que indica si el num anterior es un cero
        cells, cell = [],[]
        i = 0
        for num in antibodie:
            if (num == 0): 
                if flag == 0: 
                    i=i+1
                    cells.append(cell)
                    # print(cell)
                    cell = []
                flag = 1
            else: 
                cell.append(num)
                # print(num)
                flag = 0
                if num == antibodie[len(antibodie)-1]:
                    cells.append(cell)
        total_cells.append(cells)
        # print("cells", cells)
    # print("list of all cells for all antibodies",total_cells)
    # print("cells for antibody 1",total_cells[0])
    return total_cells

In [ ]:
#PRUEBA PARA SOLVENTAR PROBLEMA DE ANTICUERPOS  QUE EMPIEZAN POR CERO
antibodies[1][0] = 0

In [34]:
total_cells = cell_identification(antibodies = antibodies)
total_cells

antibodie [ 1  8 11  0  0 12  0  9 10  0  6  2  4  7  3  5]
antibodie [ 0  9  6  3  4  2  7 10  0  5 12  8  0  0 11  0]
antibodie [ 1 10 12 11  0  8  0  0  5  0  2  9  3  7  6  4]


[[[1, 8, 11], [12], [9, 10], [6, 2, 4, 7, 3, 5]],
 [[9, 6, 3, 4, 2, 7, 10], [5, 12, 8], [11]],
 [[1, 10, 12, 11], [8], [5], [2, 9, 3, 7, 6, 4]]]

reorganizar filas y columnas de la matriz maquinas-trabajos en función de lo descrito por el anticuerpo

In [9]:
def decode_cells(total_cells):
    total_machines, total_parts = [], []
    for antibodie in total_cells:
        print("antibodie",antibodie)
        machines = []
        parts = []
        decoded_antibodie = antibodie
        for i in range(0,len(antibodie)):
            # print(antibodie[i])
            for j in range(0,len(antibodie[i])):
                if antibodie[i][j] <= p: 
                    parts.append(rows[antibodie[i][j]-1])
                    decoded_antibodie[i][j] = rows[antibodie[i][j]-1]
                else: 
                    machines.append(columns[antibodie[i][j]-p-1])
                    decoded_antibodie[i][j] = columns[antibodie[i][j]-p-1]
            antibodie = decoded_antibodie
        total_machines.append(machines)
        total_parts.append(parts)
        print("decoded",antibodie)

    return total_machines, total_parts
    # print(total_machines)
    # print(total_parts)

In [10]:
total_machines, total_parts = decode_cells(total_cells=total_cells)

antibodie [[1, 8, 11], [12], [9, 10], [6, 2, 4, 7, 3, 5]]
decoded [['P1', 'M1', 'M4'], ['M5'], ['M2', 'M3'], ['P6', 'P2', 'P4', 'P7', 'P3', 'P5']]
antibodie [[1, 9, 6, 3, 4, 2, 7, 10], [5, 12, 8], [11]]
decoded [['P1', 'M2', 'P6', 'P3', 'P4', 'P2', 'P7', 'M3'], ['P5', 'M5', 'M1'], ['M4']]
antibodie [[1, 10, 12, 11], [8], [5], [2, 9, 3, 7, 6, 4]]
decoded [['P1', 'M3', 'M5', 'M4'], ['M1'], ['P5'], ['P2', 'M2', 'P3', 'P7', 'P6', 'P4']]


Representación de la matriz: usamos total_machines y total_parts, donde hemos colocado en orden las máquinas y los trabajos respectivamente.


In [11]:
def create_machine_part_matrix(matrix, antibodies, total_machines, total_parts):
    antibody_matrices = []
    for i in range(0,len(antibodies)):
        antibodie_matrix = matrix.loc[:,total_machines[i]]
        antibodie_matrix = antibodie_matrix.loc[total_parts[i]]
        antibody_matrices.append(antibodie_matrix)
    return antibody_matrices


antibody_matrices = create_machine_part_matrix(matrix=matrix,
                                                antibodies=antibodies, 
                                                total_machines=total_machines, 
                                                total_parts=total_parts)
antibody_matrices

[    M1  M4  M5  M2  M3
 P1   1   1   0   0   0
 P6   1   0   1   0   1
 P2   0   1   1   1   0
 P4   0   1   1   1   1
 P7   1   0   0   0   0
 P3   0   1   0   1   1
 P5   1   0   1   1   1,
     M2  M3  M5  M1  M4
 P1   0   0   0   1   1
 P6   0   1   1   1   0
 P3   1   1   0   0   1
 P4   1   1   1   0   1
 P2   1   0   1   0   1
 P7   0   0   0   1   0
 P5   1   1   1   1   0,
     M3  M5  M4  M1  M2
 P1   0   0   1   1   0
 P5   1   1   0   1   1
 P2   0   1   1   0   1
 P3   1   0   1   0   1
 P7   0   0   0   1   0
 P6   1   1   0   1   0
 P4   1   1   1   0   1]

In [12]:
def evaluate_antibodies(antibody_matrices, total_cells): #m and p should be added as parameters
    exceptions, voids = [],[]
    for i in range(0,len(total_cells)):
        # print("\n",total_cells[i])
        # print(antibody_matrices[i])
        void, exception = 0,0
        for cell in total_cells[i]:
            # print(cell)
            machines, parts = [], []
            for mp in cell:
                if mp[0] == 'M': 
                    machines.append(mp)
                    # print(machines)
                if mp[0] == 'P': 
                    parts.append(mp)
                    # print(parts)
                # else: print('error')
            
            for machine in machines:
                for part in antibody_matrices[i].index:
                    if part in parts and antibody_matrices[i][machine][part] == 0: 
                        void = void+1
                        # print("void",machine, part)
                    if part not in parts and antibody_matrices[i][machine][part] == 1:
                        exception = exception+1
                        # print("exception",machine, part)
            # print(void, exception)
            # print("\n")
        voids.append(void)
        exceptions.append(exception)
    # print("voids",voids)
    # print("exceptions",exceptions)
    efficacies = []
    matrix_dimension = m*p
    for i in range(0,len(total_cells)):
        exceptions_ratio = exceptions[i]/matrix_dimension
        voids_ratio = voids[i]/matrix_dimension
        efficacy= (1-exceptions_ratio)/(1+voids_ratio)
        efficacies.append(efficacy)
    return efficacies, voids, exceptions


In [13]:
efficacies, voids, exceptions = evaluate_antibodies(antibody_matrices = antibody_matrices, total_cells=total_cells)
efficacies

[0.48571428571428577, 0.5609756097560975, 0.4871794871794872]

In [14]:
def calculate_efficacies(total_cells, m, p):
    efficacies = []
    matrix_dimension = m*p
    for i in range(0,len(total_cells)):
        exceptions_ratio = exceptions[i]/matrix_dimension
        voids_ratio = voids[i]/matrix_dimension
        efficacy= (1-exceptions_ratio)/(1+voids_ratio)
        efficacies.append(efficacy)
    return efficacies 

In [15]:
efficacies = calculate_efficacies(total_cells = total_cells, m = m, p = p)
efficacies

[0.48571428571428577, 0.5609756097560975, 0.4871794871794872]

# Select N% of antibodies with highest affinities & Clone selected antibodies

con el parámetro p de probabilities de np.random.choice podemos pasar un vector de probabilidades para el sampleo

In [16]:
def antibodies_selection(antibodies, N):
    positions_antibodies_selected = np.random.choice(len(antibodies), size=(round(len(antibodies)*N)), replace=False)
    antibodies_selected = antibodies[positions_antibodies_selected.tolist()]
    return antibodies_selected, positions_antibodies_selected

cloned_antibodies, positions_antibodies_selected = antibodies_selection(antibodies=antibodies, N=0.66)

print(positions_antibodies_selected)
cloned_antibodies

[1 2]


array([[ 1,  9,  6,  3,  4,  2,  7, 10,  0,  5, 12,  8,  0,  0, 11,  0],
       [ 1, 10, 12, 11,  0,  8,  0,  0,  5,  0,  2,  9,  3,  7,  6,  4]])

# Mutation operator

Aqui estoy comentiendo un fallo y es que realizo la mutación en todos los casos.
Se debe comparar la efficacy del anticuerpo antes y después de la mutación y quedarse con el mejor.

## Maturate cloned antibodies

In [17]:
def mutate_cloned_antibodies(cloned_antibodies):
    for antibodie in cloned_antibodies:
        print(antibodie)
        positions = np.random.choice(len(antibodie),size=2,replace=False)
        print(positions)
        antibodie[positions[0]], antibodie[positions[1]] = antibodie[positions[1]], antibodie[positions[0]]
    return cloned_antibodies

In [18]:
mutate_cloned_antibodies(cloned_antibodies = cloned_antibodies)

[ 1  9  6  3  4  2  7 10  0  5 12  8  0  0 11  0]
[ 6 15]
[ 1 10 12 11  0  8  0  0  5  0  2  9  3  7  6  4]
[13  9]


array([[ 1,  9,  6,  3,  4,  2,  0, 10,  0,  5, 12,  8,  0,  0, 11,  7],
       [ 1, 10, 12, 11,  0,  8,  0,  0,  5,  7,  2,  9,  3,  0,  6,  4]])

## Evaluate cloned antibodies

In [19]:
cloned_total_cells = cell_identification(cloned_antibodies)
cloned_total_cells

antibodie [ 1  9  6  3  4  2  0 10  0  5 12  8  0  0 11  7]
antibodie [ 1 10 12 11  0  8  0  0  5  7  2  9  3  0  6  4]


[[[1, 9, 6, 3, 4, 2], [10], [5, 12, 8], [11, 7]],
 [[1, 10, 12, 11], [8], [5, 7, 2, 9, 3], [6, 4]]]

In [20]:
cloned_total_machines, cloned_total_parts = decode_cells(cloned_total_cells)

antibodie [[1, 9, 6, 3, 4, 2], [10], [5, 12, 8], [11, 7]]
decoded [['P1', 'M2', 'P6', 'P3', 'P4', 'P2'], ['M3'], ['P5', 'M5', 'M1'], ['M4', 'P7']]
antibodie [[1, 10, 12, 11], [8], [5, 7, 2, 9, 3], [6, 4]]
decoded [['P1', 'M3', 'M5', 'M4'], ['M1'], ['P5', 'P7', 'P2', 'M2', 'P3'], ['P6', 'P4']]


In [21]:
cloned_antibody_matrices = create_machine_part_matrix(matrix = matrix,
                                                        antibodies=cloned_antibodies, 
                                                        total_machines = cloned_total_machines, 
                                                        total_parts= cloned_total_parts)
cloned_antibody_matrices

[    M2  M3  M5  M1  M4
 P1   0   0   0   1   1
 P6   0   1   1   1   0
 P3   1   1   0   0   1
 P4   1   1   1   0   1
 P2   1   0   1   0   1
 P5   1   1   1   1   0
 P7   0   0   0   1   0,
     M3  M5  M4  M1  M2
 P1   0   0   1   1   0
 P5   1   1   0   1   1
 P7   0   0   0   1   0
 P2   0   1   1   0   1
 P3   1   0   1   0   1
 P6   1   1   0   1   0
 P4   1   1   1   0   1]

In [22]:
cloned_efficacies, cloned_voids, cloned_exceptions = evaluate_antibodies(cloned_antibody_matrices, cloned_total_cells)
cloned_efficacies

[0.5263157894736842, 0.5000000000000001]

## Add R% of best cloned antibodies to the pool of antibodies

select the R% of best cloned antibodies and adds them to the pool of antibodies

In [23]:
def select_best_cloned_antibodies(antibodies, cloned_antibodies, efficacies, cloned_efficacies, R):
    updated_efficacies = 0
    amount_antibodies = round(len(cloned_efficacies)*R)
    print("{} antibodies were added to the antibodies pool".format(amount_antibodies))
    positions = [i
        for x, i
        in heapq.nlargest(
            amount_antibodies,
            ((x, i) for i, x in enumerate(cloned_efficacies)))]
    updated_antibodies_pool = np.concatenate((antibodies, cloned_antibodies[(positions)]), axis = 0)
    cloned_efficacies = np.array(cloned_efficacies)
    updated_efficacies = efficacies + cloned_efficacies[(positions)].tolist()
    return updated_antibodies_pool, updated_efficacies

In [24]:
antibodies_pool, updated_efficacies = select_best_cloned_antibodies(antibodies = antibodies,
                                                                    cloned_antibodies = cloned_antibodies,
                                                                    efficacies = efficacies, cloned_efficacies = cloned_efficacies, R = 0.5)
print(updated_efficacies)
antibodies_pool


1 antibodies were added to the antibodies pool
[0.48571428571428577, 0.5609756097560975, 0.4871794871794872, 0.5263157894736842]


array([[ 1,  8, 11,  0,  0, 12,  0,  9, 10,  0,  6,  2,  4,  7,  3,  5],
       [ 1,  9,  6,  3,  4,  2,  7, 10,  0,  5, 12,  8,  0,  0, 11,  0],
       [ 1, 10, 12, 11,  0,  8,  0,  0,  5,  0,  2,  9,  3,  7,  6,  4],
       [ 1,  9,  6,  3,  4,  2,  0, 10,  0,  5, 12,  8,  0,  0, 11,  7]])

# Remove worst members of the antibodies pool (RECEPTOR EDITING)

"After mutation processes, the
antibodies that have worse efficacy values are erased (worst
%B of the whole population)"

In [25]:
def receptor_editing(antibodies_pool,updated_efficacies, B):
    amount_antibodies = round(len(updated_efficacies)*B)
    print("Se han borrado {} anticuerpos".format(amount_antibodies))
    positions = [i
        for x, i
        in heapq.nsmallest(
            amount_antibodies,
            ((x, i) for i, x in enumerate(updated_efficacies)))]
    print(positions)
    antibodies_pool = np.delete(antibodies_pool, positions, axis=0)
    return antibodies_pool

In [26]:
B = 0.5
antibodies_pool = receptor_editing(antibodies_pool = antibodies_pool, updated_efficacies = updated_efficacies, B = B)
antibodies_pool

Se han borrado 2 anticuerpos
[0, 2]


array([[ 1,  9,  6,  3,  4,  2,  7, 10,  0,  5, 12,  8,  0,  0, 11,  0],
       [ 1,  9,  6,  3,  4,  2,  0, 10,  0,  5, 12,  8,  0,  0, 11,  7]])

# New Random antibodies into the population

"Then, same percent of new
antibodies are randomly generated"

La pregunta es la misma cantidad de anticuerpos que hemos borrado en el apartado anterior o el mismo porcentaje (B) sobre la nueva población que se ha visto disminuida por el receptor editing.

In [27]:
number_new_random_antibodies = round(len(antibodies_pool)*B)

new_random_antibodies = generation_initial_population(p = p, 
                                                    m = m, 
                                                    population_size = number_new_random_antibodies, 
                                                    seed = 1995)
print(new_random_antibodies)

antibodies_pool = np.concatenate((antibodies_pool, new_random_antibodies), axis = 0)
antibodies_pool

[[ 1  8 11  0  0 12  0  9 10  0  6  2  4  7  3  5]]


array([[ 1,  9,  6,  3,  4,  2,  7, 10,  0,  5, 12,  8,  0,  0, 11,  0],
       [ 1,  9,  6,  3,  4,  2,  0, 10,  0,  5, 12,  8,  0,  0, 11,  7],
       [ 1,  8, 11,  0,  0, 12,  0,  9, 10,  0,  6,  2,  4,  7,  3,  5]])

# Stopping criteria